In [62]:
import pandas as pd
from typing import Optional, List
from core import system_msg, user_msg, assistant_msg, generic_call_stream
from qdrant_search import QdrantSearchEngine

In [63]:
from psql_models import get_session

In [64]:
engine = QdrantSearchEngine()

In [65]:
#await engine.initialize()
await engine.sync_from_postgres(get_session)

Created text index for field: topic_name
Created text index for field: summary
Created text index for field: details


In [66]:
pd.options.display.max_colwidth = 1000

In [111]:
class ChatSession:
    def __init__(self, search_engine: QdrantSearchEngine, prompts):
        self.search_engine = search_engine
        self.prompts = prompts
        self.messages: List = []
        self.results = None
    
    def add_message(self, role: str, content: str):
        """Add a message to the conversation history"""
        if role == "system":
            self.messages.append(system_msg(content))
        elif role == "user":
            self.messages.append(user_msg(content))
        elif role == "assistant":
            self.messages.append(assistant_msg(content))
    
    async def process_user_message(self, query: str):
        """Process a user message and generate a response"""
        self.add_message("user", query)
        
        # Split query and instructions if present
        query, instructions = query.split(':') if ':' in query else (query, '')
        
        # Search for context using both methods
        main_results = await self.search_engine.search(
            query_text=query,
            limit=200,
            min_score=0.4,
        )
        
        speaker_results = await self.search_engine.search_by_speaker(
            speaker_query=query,
            limit=200,
            min_score=0.49
        )
        
        # Convert results to DataFrames
        main_df = pd.DataFrame(main_results) if main_results else pd.DataFrame()
        speaker_df = pd.DataFrame(speaker_results) if speaker_results else pd.DataFrame()
        
        # Select relevant columns and combine results
        columns = ['topic_name', 'speaker_name', 'summary', 'details']
        score_columns = ['score', 'vector_scores', 'exact_matches']
        
        if len(main_df) > 0:
            main_df = main_df[columns + score_columns]
            main_df['source'] = 'main'
        else:
            main_df = pd.DataFrame(columns=columns + score_columns + ['source'])
        
        if len(speaker_df) > 0:
            speaker_df = speaker_df[columns + ['score']]  # Speaker search has simpler scoring
            speaker_df['source'] = 'speaker'
        else:
            speaker_df = pd.DataFrame(columns=columns + ['score', 'source'])
        
        # Combine, deduplicate and sort results
        self.results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)
        if not self.results.empty:
            self.results = self.results.sort_values('score', ascending=False)

        # Create context from results
        context = self.results[columns].to_markdown(index=False) if not self.results.empty else "No relevant context found."
        
        # Add context message
        context_msg = system_msg(f"Context: {context}")
        
        # Generate response
        messages_with_context = [
            system_msg(self.prompts.perplexity),
            system_msg('special instructions: ' + instructions),
            *self.messages,
            context_msg
        ]
        
        response = await generic_call_stream(messages_with_context)
        
        # Add assistant response
        self.add_message("assistant", response)
        
        return response

    def get_results(self) -> Optional[pd.DataFrame]:
        """Get the latest search results"""
        return self.results

In [112]:
# Initialize
from prompts import Prompts
prompts = Prompts()
search_engine = QdrantSearchEngine()
chat = ChatSession(search_engine, prompts)

In [113]:

# Process a message
response = await chat.process_user_message("alexey")

Alexey is referenced in various contexts related to software development and project management. He is responsible for providing a new build aimed at resolving issues with an extension, particularly concerning button functionality[1][4]. Additionally, he is involved in creating the user interface for a project, where his ideas about the assistant's functionality need clarification[3][5]. 

In another context, Alexey Khintz is mentioned as a contact in the Web3 space, potentially offering insights into audience preferences[2]. Olga Nemirovskaya suggested leveraging Alexey's network for promoting a project called Vexu, indicating his influence and connections within the industry[3].


In [114]:
from IPython.display import Markdown
Markdown(response)

Alexey is referenced in various contexts related to software development and project management. He is responsible for providing a new build aimed at resolving issues with an extension, particularly concerning button functionality[1][4]. Additionally, he is involved in creating the user interface for a project, where his ideas about the assistant's functionality need clarification[3][5]. 

In another context, Alexey Khintz is mentioned as a contact in the Web3 space, potentially offering insights into audience preferences[2]. Olga Nemirovskaya suggested leveraging Alexey's network for promoting a project called Vexu, indicating his influence and connections within the industry[3].

In [117]:
chat.results[chat.results['source']=='speaker']

,topic_name,speaker_name,summary,details,score,vector_scores,exact_matches,source
80,Tools for Recording and Editing,Alex Shevliakov,"They explored tools for recording and editing promotional content, specifically mentioning Screen Studio.","Alex shared his experience with Screen Studio, which automates cursor tracking and enhances the quality of promotional videos.",0.592849,NaN,NaN,speaker
89,ассистент,Alex Shevliakov,The assistant is discussed as a tool for user interaction within the dashboard.,The team is considering how the assistant can help users filter and interact with call data effectively.,0.592849,NaN,NaN,speaker
88,платежи,Alex Shevliakov,Payments are discussed in relation to the dashboard's functionalities.,"Alex mentioned the need to integrate payment methods, specifically referencing Stripe.",0.592849,NaN,NaN,speaker
87,Screen Studio,Alex Shevliakov,Screen Studio is a tool used for creating promotional videos and screen recordings.,"Alex mentions using Screen Studio for its features that enhance video quality, such as tracking the cursor and creating smooth transitions.",0.592849,NaN,NaN,speaker
86,Need for more working hours,Alex Shevliakov,"AS expressed the need to increase working hours to manage a larger workload effectively, indicating that 25 hours are insufficient for the tasks at hand.","AS mentioned that the workload has increased significantly, and he feels that he needs to allocate more hours to complete the necessary tasks. No specific deadline was mentioned, but the urgency is implied due to the competitive nature of the work.",0.592849,NaN,NaN,speaker
85,Next Steps and Deadlines,Alex Shevliakov,"The team outlined the next steps for development, including creating a separate branch for changes and setting deadlines for deliverables.","Alex Shevliakov plans to start working on the dashboard improvements by Saturday, allocating 2-3 hours initially, with additional time next week. He will keep the team updated on progress weekly.",0.592849,NaN,NaN,speaker
84,Upwork,Alex Shevliakov,Upwork is a freelancing platform discussed as a less favorable option for hiring freelancers.,"Alex shares his negative experiences with hiring freelancers from Upwork, recommending against it.",0.592849,NaN,NaN,speaker
83,Logo and Branding Elements,Alex Shevliakov,The need for a logo and branding elements was discussed to enhance the product's identity.,Alex will look into creating a logo and branding elements that align with the product's design and marketing strategy.,0.592849,NaN,NaN,speaker
82,Next.js,Alex Shevliakov,"Next.js is a modern framework for React applications, discussed as a better alternative for development.",Alex suggests using Next.js to improve the efficiency of the project and reduce development time.,0.592849,NaN,NaN,speaker
81,VEX,Alex Shevliakov,VEX is a project mentioned by Alex Shevliakov that is currently demanding a lot of his time.,"Alex is balancing his workload with VEX and other projects, indicating its significance in his current responsibilities.",0.592849,NaN,NaN,speaker
